In [159]:
import pandas as pd
import sqlalchemy, os, re, pymysql
from datetime import datetime, timedelta
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

In [160]:
load_dotenv('config.env')
username = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
database_name = os.getenv("DB_NAME")

connect = f"mysql+pymysql://{username}:{password}@{host}/{database_name}"
engine = sqlalchemy.create_engine(connect)
conn = engine.connect()

In [161]:
def sql_to_dataframe(table, engine):
    q = f'SELECT * FROM {table}'
    return pd.read_sql(q, engine)

user_df = sql_to_dataframe('user', engine=engine)
exercise_set_df = sql_to_dataframe('exercise_set', engine=engine)
common_user_ids = pd.Index(user_df['id']).intersection(exercise_set_df['user_id'])

filtered_user_df = user_df[user_df['id'].isin(common_user_ids)]
filtered_exercise_set_df = exercise_set_df[exercise_set_df['user_id'].isin(common_user_ids)]


In [166]:
class PumpX_Transform():

    def __init__(self, user_df, exercise_set_df) -> None:
        self.user_df = user_df
        self.exercise_set_df = exercise_set_df

    def get_total_loggings_week(self):
        result_dataframe = self.exercise_set_df
        result_dataframe['approach_time'] = pd.to_datetime(result_dataframe['approach_time'], errors='coerce')
        result_dataframe.dropna(subset=['approach_time'], inplace=True)
        current_week = pd.Timestamp.now().isocalendar()[1]
        result_dataframe['week_number'] = result_dataframe['approach_time'].apply(lambda x: x.isocalendar()[1])
        current_year = pd.Timestamp.now().year
        result_dataframe['year'] = result_dataframe['approach_time'].apply(lambda x: x.isocalendar()[0])
        filtered_df = result_dataframe[(result_dataframe['week_number'] == current_week) & (result_dataframe['year'] == current_year)]
        result = filtered_df.groupby(filtered_df['approach_time'].dt.date)['user_id'].nunique().reset_index()
        result.columns = ['logging_date', 'total_loggings']
        return result
    
    def get_goal_and_age_distribution(self):
        labels = ['under_20', 'age_20_29', 'age_30_39', 'age_40_49', 'age_50_59', 'age_60_above']
        result_dataframe = self.user_df[(self.user_df['goal'].notna()) & (self.user_df['age'].notna())]
        bins = [0, 20, 30, 40, 50, 60, 100]
        result_dataframe['age_group'] = pd.cut(result_dataframe['age'], bins=bins, labels=labels, right=False)
        return result_dataframe.groupby(['goal', 'age_group'])['id'].nunique().unstack(fill_value=0).reset_index()
    
    def get_logging_usage_distribution(self):
        copy_dataframe = self.exercise_set_df
        copy_dataframe['approach_time'] = pd.to_datetime(copy_dataframe['approach_time'])
        result = copy_dataframe.groupby('user_id').agg(
            last_date_logged=('approach_time', 'max'),
            count_logged=('user_id', 'count')
        ).reset_index()

        result['last_date_logged'] = pd.to_datetime(result['last_date_logged'])
        now = datetime.now()
        curr_week = now - timedelta(days = now.weekday())
        end_curr_week = curr_week + timedelta(days = 6)
        result = result[(result['last_date_logged'] >= curr_week) & (result['last_date_logged'] <= end_curr_week)]
        return result

        # filtered_week_df = result[(result['last_date_logged'] >= curr_week) & (result['last_date_logged'] <= end_curr_week) ]

        # result['week'] = result['last_date_logged'].str.dt.isocalender().week
        # result['day'] = result['last_date_logged'].dt.isocalender().date
        # result['year'] = result['last_date_logged'].dt.isocalender().year
        # current_week = pd.Timestamp.now().isocalendar().week
        # current_year = pd.Timestamp.now().isocalendar().year
        # current_week_df = result[(exercise_set_df['week'] == current_week) & (result['year'] == current_year)]
        # return filtered_week_df

analytics_prod = PumpX_Transform(filtered_user_df, filtered_exercise_set_df)
analytics_prod.get_logging_usage_distribution()

# loggings_by_week = analytics_prod.get_total_loggings_week()

,user_id,last_date_logged,count_logged
36,1861,2024-05-16 15:44:32,513
48,1896,2024-05-16 17:39:49,1527
82,2004,2024-05-15 22:00:52,2145
89,2022,2024-05-17 08:33:10,1897
93,2041,2024-05-15 18:44:06,1447
...,...,...,...
539,4175,2024-05-16 16:46:31,11
540,4183,2024-05-17 14:22:39,4
541,4184,2024-05-17 20:55:43,13
542,4193,2024-05-18 18:37:45,20
